For the prediction of the outcome of the TEAM A X TEAM B game, we will feed the model with the stats of the mean past 10 games of team A at HOME and the mean past 10 games of team B away

Features description

    ### Basic Box Score Stats




    MP -- Minutes Played

    FG -- Field Goals

    FGA -- Field Goal Attempts

    FG% -- Field Goal Percentage

    3P -- 3-Point Field Goals

    3PA -- 3-Point Field Goal Attempts

    3P% -- 3-Point Field Goal Percentage

    FT -- Free Throws

    FTA -- Free Throw Attempts

    FT% -- Free Throw Percentage

    ORB -- Offensive Rebounds

    DRB -- Defensive Rebounds

    TRB -- Total Rebounds
    AST -- Assists

    STL -- Steals

    BLK -- Blocks

    TOV -- Turnovers

    PF -- Personal Fouls

    PTS -- Points

    +/- -- Plus/Minus

    MP -- Minutes Played

    FG -- Field Goals

    FGA -- Field Goal Attempts

    FG% -- Field Goal Percentage

    3P -- 3-Point Field Goals

    3PA -- 3-Point Field Goal Attempts

    3P% -- 3-Point Field Goal Percentage

    FT -- Free Throws

    FTA -- Free Throw Attempts

    FT% -- Free Throw Percentage

    ORB -- Offensive Rebounds

    DRB -- Defensive Rebounds

    TRB -- Total Rebounds

    AST -- Assists

    STL -- Steals

    BLK -- Blocks

    TOV -- Turnovers

    PF -- Personal Fouls

    PTS -- Points

    +/- -- Plus/Minus


    ### Advanced Box Score Stats

    MP -- Minutes Played

    TS% -- True Shooting Percentage
    A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.

    eFG% -- Effective Field Goal Percentage
    This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.

    3PAr -- 3-Point Attempt Rate
    Percentage of FG Attempts from 3-Point Range

    FTr -- Free Throw Attempt Rate
    Number of FT Attempts Per FG Attempt

    ORB% -- Offensive Rebound Percentage
    An estimate of the percentage of available offensive rebounds a player grabbed while they were on the floor.

    DRB% -- Defensive Rebound Percentage
    An estimate of the percentage of available defensive rebounds a player grabbed while they were on the floor.

    TRB% -- Total Rebound Percentage
    An estimate of the percentage of available rebounds a player grabbed while they were on the floor.

    AST% -- Assist Percentage
    An estimate of the percentage of teammate field goals a player assisted while they were on the floor.

    STL% -- Steal Percentage
    An estimate of the percentage of opponent possessions that end with a steal by the player while they were on the floor.

    BLK% -- Block Percentage
    An estimate of the percentage of opponent two-point field goal attempts blocked by the player while they were on the floor.

    TOV% -- Turnover Percentage
    An estimate of turnovers committed per 100 plays.

    USG% -- Usage Percentage
    An estimate of the percentage of team plays used by a player while they were on the floor.

    ORtg -- Offensive Rating
    An estimate of points produced (players) or scored (teams) per 100 possessions

    DRtg -- Defensive Rating
    An estimate of points allowed per 100 possessions

    BPM -- Box Plus/Minus
    A box score estimate of the points per 100 possessions a player contributed above a league-average player, translated to an average team.

    MP -- Minutes Played

    TS% -- True Shooting Percentage
    A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.

    eFG% -- Effective Field Goal Percentage
    This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.

    3PAr -- 3-Point Attempt Rate
    Percentage of FG Attempts from 3-Point Range

    FTr -- Free Throw Attempt Rate
    Number of FT Attempts Per FG Attempt

    ORB% -- Offensive Rebound Percentage
    An estimate of the percentage of available offensive rebounds a player grabbed while they were on the floor.

    DRB% -- Defensive Rebound Percentage
    An estimate of the percentage of available defensive rebounds a player grabbed while they were on the floor.

    TRB% -- Total Rebound Percentage
    An estimate of the percentage of available rebounds a player grabbed while they were on the floor.

    AST% -- Assist Percentage
    An estimate of the percentage of teammate field goals a player assisted while they were on the floor.

    STL% -- Steal Percentage
    An estimate of the percentage of opponent possessions that end with a steal by the player while they were on the floor.

    BLK% -- Block Percentage
    An estimate of the percentage of opponent two-point field goal attempts blocked by the player while they were on the floor.

    TOV% -- Turnover Percentage
    An estimate of turnovers committed per 100 plays.

    USG% -- Usage Percentage
    An estimate of the percentage of team plays used by a player while they were on the floor.

    ORtg -- Offensive Rating
    An estimate of points produced (players) or scored (teams) per 100 possessions

    DRtg -- Defensive Rating
    An estimate of points allowed per 100 possessions

    BPM -- Box Plus/Minus
    A box score estimate of the points per 100 possessions a player contributed above a league-average player, translated to an average team.

In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy, sqlite3
import os
import warnings

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import make_pipeline

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

We first gonna extract our stored data from the database created on the webscraping step.

In [140]:
engine = sqlalchemy.create_engine('sqlite:///nba_games.db', echo=False)

df = pd.read_sql('SELECT * FROM nba_games', con=engine)

Exploring data

In [141]:
df.head()

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,opponent_tov%_max,opponent_usg%_max,opponent_ortg_max,opponent_drtg_max,opponent_Team,opponent_Total,opponent_season,opponent_date,opponent_home,WIN
0,240.0,240.0,37.0,99.0,0.374,10.0,33.0,0.303,15.0,23.0,...,43.6,40.4,300.0,98.0,BRK,125,2020-21,2020-12-22,1,0
1,240.0,240.0,42.0,92.0,0.457,15.0,35.0,0.429,26.0,32.0,...,50.0,32.1,267.0,120.0,GSW,99,2020-21,2020-12-22,0,1
2,240.0,240.0,44.0,93.0,0.473,14.0,40.0,0.350,14.0,19.0,...,100.0,35.9,166.0,118.0,LAL,109,2020-21,2020-12-22,1,1
3,240.0,240.0,38.0,81.0,0.469,9.0,29.0,0.310,24.0,31.0,...,20.9,40.2,154.0,114.0,LAC,116,2020-21,2020-12-22,0,0
4,240.0,240.0,46.0,90.0,0.511,14.0,35.0,0.400,15.0,18.0,...,33.3,39.2,203.0,126.0,BOS,122,2020-21,2020-12-23,1,0


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4984 entries, 0 to 4983
Columns: 152 entries, mp to WIN
dtypes: float64(134), int64(6), object(12)
memory usage: 5.8+ MB


In [143]:
df.columns.to_list()

['mp',
 'mp.1',
 'fg',
 'fga',
 'fg%',
 '3p',
 '3pa',
 '3p%',
 'ft',
 'fta',
 'ft%',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 '+/-',
 'ts%',
 'efg%',
 '3par',
 'ftr',
 'orb%',
 'drb%',
 'trb%',
 'ast%',
 'stl%',
 'blk%',
 'tov%',
 'usg%',
 'ortg',
 'drtg',
 'mp_max',
 'mp_max.1',
 'fg_max',
 'fga_max',
 'fg%_max',
 '3p_max',
 '3pa_max',
 '3p%_max',
 'ft_max',
 'fta_max',
 'ft%_max',
 'orb_max',
 'drb_max',
 'trb_max',
 'ast_max',
 'stl_max',
 'blk_max',
 'tov_max',
 'pf_max',
 'pts_max',
 '+/-_max',
 'ts%_max',
 'efg%_max',
 '3par_max',
 'ftr_max',
 'orb%_max',
 'drb%_max',
 'trb%_max',
 'ast%_max',
 'stl%_max',
 'blk%_max',
 'tov%_max',
 'usg%_max',
 'ortg_max',
 'drtg_max',
 'Team',
 'Total',
 'season',
 'date',
 'home',
 'opponent_index',
 'opponent_mp',
 'opponent_mp.1',
 'opponent_fg',
 'opponent_fga',
 'opponent_fg%',
 'opponent_3p',
 'opponent_3pa',
 'opponent_3p%',
 'opponent_ft',
 'opponent_fta',
 'opponent_ft%',
 'opponent_orb',
 'opponent_drb',

We are going to delete some non useful or redundant info, as well as the rows where the number of minutes played != 240, because these are some outliers, and we will then remove the  mp variable alltogheter

In [144]:
len(df[df['mp'] != 240])/len(df)

0.047752808988764044

In [145]:
print('porcentagem de outliers: {:.2%}'.format(len(df[df['mp'] != 240])/len(df)))

    
df = df[df['mp'] == 240]
df.shape

porcentagem de outliers: 4.78%


(4746, 152)

In [146]:
del_list = ['mp.1','+/-','mp_max','mp_max.1',
 'opponent_index','opponent_mp',
 'opponent_mp.1','opponent_+/-','opponent_mp_max.1',
  'opponent_Team', 'opponent_date', 'opponent_home', 'opponent_season']


for col in del_list:
    if col in df.columns:
        del df[col]

In [147]:
X = df.loc[:, df.columns != 'WIN']
y = df['WIN']

There are some usefull columns that we wont use for tranining, but for indexing. Date and teamwill be useful for knowing the past 10 games of a team, and season will be usefull if we decide to train data per season

In [148]:
info_cols = ['date', 'Team', 'season']
model_cols = X.columns[~X.columns.isin(info_cols)]	

In [149]:
X[model_cols].columns.to_list()

['mp',
 'fg',
 'fga',
 'fg%',
 '3p',
 '3pa',
 '3p%',
 'ft',
 'fta',
 'ft%',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'ts%',
 'efg%',
 '3par',
 'ftr',
 'orb%',
 'drb%',
 'trb%',
 'ast%',
 'stl%',
 'blk%',
 'tov%',
 'usg%',
 'ortg',
 'drtg',
 'fg_max',
 'fga_max',
 'fg%_max',
 '3p_max',
 '3pa_max',
 '3p%_max',
 'ft_max',
 'fta_max',
 'ft%_max',
 'orb_max',
 'drb_max',
 'trb_max',
 'ast_max',
 'stl_max',
 'blk_max',
 'tov_max',
 'pf_max',
 'pts_max',
 '+/-_max',
 'ts%_max',
 'efg%_max',
 '3par_max',
 'ftr_max',
 'orb%_max',
 'drb%_max',
 'trb%_max',
 'ast%_max',
 'stl%_max',
 'blk%_max',
 'tov%_max',
 'usg%_max',
 'ortg_max',
 'drtg_max',
 'Total',
 'home',
 'opponent_fg',
 'opponent_fga',
 'opponent_fg%',
 'opponent_3p',
 'opponent_3pa',
 'opponent_3p%',
 'opponent_ft',
 'opponent_fta',
 'opponent_ft%',
 'opponent_orb',
 'opponent_drb',
 'opponent_trb',
 'opponent_ast',
 'opponent_stl',
 'opponent_blk',
 'opponent_tov',
 'opponent_pf',
 'opponent_pts',
 'op

As we have many columns, we will run the feature selection method to extract only features that will be meaningful for training the model

We are going to use with this method , the TimeSeriesSplit method, because we dont want just to split totaly randonly the data for training and validation, as we want to use accordnly with the period of time. For example, I can use this to train against data per season.

In [150]:
scaler = MinMaxScaler()
clf = make_pipeline(scaler, LinearSVC(random_state=42, C=1, tol=1e-5))

split = TimeSeriesSplit(n_splits=5)
selector = SequentialFeatureSelector(clf, n_features_to_select=40, cv=split, n_jobs=-1, direction='forward')


In [151]:
X_transf = scaler.fit_transform(X[model_cols])

c:\Users\Usuario\anaconda3\envs\nba\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\Usuario\anaconda3\envs\nba\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [152]:
X[model_cols]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,opponent_drb%_max,opponent_trb%_max,opponent_ast%_max,opponent_stl%_max,opponent_blk%_max,opponent_tov%_max,opponent_usg%_max,opponent_ortg_max,opponent_drtg_max,opponent_Total
0,240.0,37.0,99.0,0.374,10.0,33.0,0.303,15.0,23.0,0.652,...,44.5,29.8,33.0,7.3,7.4,43.6,40.4,300.0,98.0,125
1,240.0,42.0,92.0,0.457,15.0,35.0,0.429,26.0,32.0,0.813,...,50.1,29.7,61.1,3.7,12.7,50.0,32.1,267.0,120.0,99
2,240.0,44.0,93.0,0.473,14.0,40.0,0.350,14.0,19.0,0.737,...,39.6,24.4,47.0,4.3,3.9,100.0,35.9,166.0,118.0,109
3,240.0,38.0,81.0,0.469,9.0,29.0,0.310,24.0,31.0,0.774,...,38.7,16.9,24.2,4.7,4.5,20.9,40.2,154.0,114.0,116
4,240.0,46.0,90.0,0.511,14.0,35.0,0.400,15.0,18.0,0.833,...,26.9,19.3,22.2,3.8,7.0,33.3,39.2,203.0,126.0,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,240.0,34.0,85.0,0.400,15.0,38.0,0.395,14.0,19.0,0.737,...,29.1,20.0,30.2,6.1,10.3,42.9,36.3,133.0,112.0,107
4980,240.0,31.0,75.0,0.413,11.0,32.0,0.344,21.0,31.0,0.677,...,31.5,17.1,59.5,5.7,7.6,33.3,36.2,222.0,107.0,104
4981,240.0,41.0,88.0,0.466,9.0,40.0,0.225,13.0,15.0,0.867,...,84.8,42.4,100.0,2.3,12.4,45.0,94.4,300.0,112.0,94
4982,240.0,38.0,92.0,0.413,19.0,46.0,0.413,8.0,8.0,1.000,...,100.0,53.8,38.7,3.9,16.0,100.0,42.6,141.0,126.0,90
